# Drawing WCS Axes with Ginga

See https://astrowidgets.readthedocs.io for additional details about the widget, including installation notes.

Also see https://ginga.readthedocs.io/en/stable/manual/plugins_local/wcsaxes.html for the Reference Viewer (standalone GUI) plugin that this notebook is based on.

In [ ]:
from astropy.nddata import CCDData
from astrowidgets import ImageWidget as _ImageWidget
from ginga.canvas.types.astro import WCSAxes
from ginga.misc.log import get_logger

In [ ]:
logger = get_logger('my viewer', log_stderr=True,
                    log_file=None, level=30)

In [ ]:
class ImageWidget(_ImageWidget):
    """Extends core widget to handle WCS axes."""
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.axes_tag = 'widget_wcsaxes'
    
    def draw_wcs_axes(self, linewidth=1, linestyle='solid', linecolor='cyan',
                      alpha=1, fontsize=8, n_ra_lines=10, n_dec_lines=10,
                      show_label=True, label_offset=4, ra_angle=None, dec_angle=None):
        """Draw or update WCS axes. Axes properties are same as those in
        https://ginga.readthedocs.io/en/stable/manual/plugins_local/wcsaxes.html ,
        except for ``ra_angle`` and ``dec_angle``.
        
        * ``linestyle`` can be 'solid' or 'dash'.
        * ``ra_angle`` and ``dec_angle`` control the angles, in degrees,
          of RA and Dec labels, respectively. In the standalone GUI plugin,
          their controls were available via plugin GUI only and not settable
          from configuration file.
        
        """
        try:
            axes = self._viewer.canvas.get_object_by_tag(self.axes_tag)
        except Exception:  # Draw new
            axes = WCSAxes(linewidth=linewidth, linestyle=linestyle, color=linecolor,
                           alpha=alpha, fontsize=fontsize)
            axes.num_ra = n_ra_lines
            axes.num_dec = n_dec_lines
            axes.show_label = show_label
            axes.txt_off = label_offset
            axes.ra_angle = ra_angle
            axes.dec_angle = dec_angle
            self._viewer.canvas.add(axes, tag=self.axes_tag)
        else:  # Update existing
            axes.linewidth = linewidth
            axes.linestyle = linestyle
            axes.color = linecolor
            axes.alpha = alpha
            axes.fontsize = fontsize
            axes.num_ra = n_ra_lines
            axes.num_dec = n_dec_lines
            axes.show_label = show_label
            axes.txt_off = label_offset
            axes.ra_angle = ra_angle
            axes.dec_angle = dec_angle
            axes.sync_state()
            w._viewer.canvas.update_canvas()

In [ ]:
w = ImageWidget(logger=logger)

For this example, we use an image from the Astropy Data Server and load it as `CCDData`. Feel free to modify `filename` to point to your desired image. It is important that the image has a valid WCS.

In [ ]:
filename = 'https://github.com/astropy/astropy-data/raw/gh-pages/galactic_center/gc_2mass_k.fits'
numhdu = 0
fluxunit = 'count'

ccd = CCDData.read(filename, hdu=numhdu, unit=fluxunit, format='fits')
print(ccd.wcs)

In [ ]:
w.load_nddata(ccd)

In [ ]:
w

In [ ]:
w.stretch = 'asinh'
w.cuts = 'histogram'
w.set_colormap('ds9_cool')

The following line draws WCS axes on the display widget above using the default settings.

In [ ]:
w.draw_wcs_axes()

This updates the WCS axes with a non-default setting.

In [ ]:
w.draw_wcs_axes(linecolor='pink', fontsize=14)